In [1]:
%load_ext autoreload
%autoreload 2

---

## Twitter Datensatz

**1. Daten sammeln**

Hier folgt in kürze noch ein beschreibender Text.Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

In [6]:
from src.data import twitter_scraper

# set search query
query = '(#technology, OR #tech, OR #innovation, OR #AI, OR #blockchain, OR #business) min_faves:1 lang:en until:2023-03-31 since:2023-01-01'

# collect twitter posts
_ = twitter_scraper.get_twitter_posts(query, limit=50)

51it [00:03, 13.15it/s]                                                                                                                                        


**2. Datensatz anzeigen**

Hier folgt in kürze noch ein beschreibender Text.Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

In [7]:
from src import utils
import pandas as pd

# load collected posts
list_of_tweets = utils.load_pkl(path='../data/raw/twitter_tweets_raw.pkl')[0]

# transform posts into a dataframe and display them
df = pd.DataFrame(list_of_tweets)
df.head(4)

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/YueDongCS/status/164159107...,2023-03-30 23:59:46+00:00,My condolences and sad that the #NLP and #AI c...,My condolences and sad that the #NLP and #AI c...,1641591075512147968,https://twitter.com/YueDongCS,0,1,4,0,...,NaN,None,None,None,None,"[NLP, AI]",None,None,916,None
1,https://twitter.com/Mlearning_ai/status/164159...,2023-03-30 23:59:43+00:00,Hiring Now: The Top Jobs of the Future Fueled ...,Hiring Now: The Top Jobs of the Future Fueled ...,1641591062757097473,https://twitter.com/Mlearning_ai,0,0,1,0,...,NaN,None,None,None,None,"[art, AIart, machinelearning, MLsoGood, artifi...",None,SummaryCard(title='Hiring Now: The Top Jobs of...,83,None
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:43+00:00,What’s the hack. #stem #science #stemeducation...,What’s the hack. #stem #science #stemeducation...,1641591062329438209,https://twitter.com/HackerAran7,0,0,3,0,...,NaN,None,None,None,None,"[stem, science, stemeducation, education, engi...",None,None,69,None
3,https://twitter.com/Stemble_/status/1641590942...,2023-03-30 23:59:14+00:00,"🚀 Mark your calendars, Apple enthusiasts! 🗓️\n...","🚀 Mark your calendars, Apple enthusiasts! 🗓️\n...",1641590942150049792,https://twitter.com/Stemble_,1,1,2,0,...,NaN,None,None,None,None,"[Apple, MixedRealityHeadset, AppleEvent, TechN...",None,None,136,None


---